Diese Version ist die unaufbereitete Version, und verwendet noch Funktionen, welche vor Abgabe verändert werden müssen!

Angepasst werden muss:
<br>[x] nogo_columns
<br>[x] rain id muss raus und an andere stelle
<br>[x] Boxenstopberechnung muss raus
<br>[x] Zielspalte muss umbenannt werden --> irgendwas mit Minuten
<br>[ ] Hyperparameteroptimierung muss fertiggestellt werden
<br>[ ] finales Testing muss rein oder in ein separates Notebook
<br>[ ]  Optimierung des p von Dropout Layer?
<br>[x] nicht nur Vorhersagen( Kommazahlen) vergleichen, sondern direkt sortieren und Podiums Positionen als klare Klassen übergeben

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import random
import operator
import pandas as pd
import numpy as np
import pprint
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
import pandasql as sqldf

Ziel dieses Notebooks ist es Klassen bereit zu stellen, welche anhand einer Grid Search orientierten Methode die Hyperparameter eines Neuronalen Netzes optimieren. 

In der ersten Klasse werden die Hyperparameter Layeranzahl und Neuronenanzahl betrachtet, während die zweite Klasse die Anzahl von Trainingsepochen und die Lernrate für den Adam Optimizer optimiert.

Für die Optimierung wird zusätzlich eine Klasse benötigt, welche dynamisch Neuronale Netze erzeugt.

In [2]:
def load_data(directory = 'sliced_data'):
    '''
        Funktion, die die aufbereiteten/vorbereiteten Daten aus existierenden CSV Dateien 
        einliest und je nach Vollständigkeit in zwei Dictionaries abspeichert,
        geschlüsselt nach der jeweiligen RaceId
    '''
    if os.path.exists(directory):
        csv_filenames = []
        #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
        for filename in os.listdir(os.getcwd()+'/'+directory):
            typ = filename.split('.')[-1]
            name = filename.split('.')[0]
            if typ == 'csv':
                csv_filenames.append(filename)
        sliced_races = {}
        #einlesen und abspeichern als dataframe aller dateien
        for file in csv_filenames:
            try:
                df = pd.read_csv(directory+'/'+file, engine = 'python', sep = ';', decimal = '.')
                del df['Unnamed: 0']
            except Exception as e:
                df = pd.read_csv(directory+'/'+file, engine = 'c', sep = ';', decimal = '.')
                del df['Unnamed: 0']
                print(e)
            f = int(file.split('_')[-1].split('.')[0]) #raceid wird als key gesetzt
            sliced_races[f] = df
        print('Einlesen der sliced Dateien erfolgreich')
    else:
        raise ('sliced Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
   
    return sliced_races

def train_dev_test(data_dict, train_p = 0.7, dev_p = 0.2, test_p = 0.1, nogo_columns = []):
    
    if round(train_p+dev_p+test_p,1) !=1.0:
        raise ValueError ('No valid train/dev/test distribution')
    
    '''
        Daten werde in einem Dictionary übergeben, Dataframes werden in dieser 
        Funktion geshuffled und dann in einen Traindatensatz, einen Development-
        datensatz und in einen Testdatensatz aufgeteilt.
    '''
    #aufteilen in train, dev, test counter
    train_count = round(len(data_dict.keys())*train_p, 0)
    dev_count = train_count+round(len(data_dict.keys())*dev_p,0)
    test_count = len(data_dict.keys())-(train_count+dev_count)
    
    #shufflen der übergebenen Daten
    keys = list(data_dict.keys())
    random.shuffle(keys)
    data_shuffled = {}
    for key in keys:
        data_shuffled[key] = data_dict[key]
        
    #erzeugen separater train,dev,test dictionaries
    train = {}
    dev = {}
    test = {}
    c = 0
    
    #daten sollen nicht in tensoren umgewandelt werden
    for id, df in data_shuffled.items():
        #entfernen nicht gewollter spalten aus dataframe
        cols = [col for col in df.columns if col not in nogo_columns]
        df = df[cols]
        if c < train_count:
            train[id] = df
        elif c >= train_count and c < dev_count:
            dev[id] = df
        else:
            test[id] = df
        c += 1
                
    return train, dev, test

def to_tensor(train_data, dev_data, test_data, nogo_columns = []):
    '''
        Funktion erhält zuvor erzeugte train, dev und test Dictionarys mit Formel 1 Rennen
        und wandelt die DateFrames in Tensoren um. Den Tensoren wird jeweils ihr Target Value
        (podium_podition) zugeordnet. In Dictionary Form werden die Tensoren zurück gegeben.
        Nicht gewollte Spalten (in nogo_columns) werden entfernt.
    '''
    train = []
    train_ = {}
    dev = []
    dev_ = {}
    test = []
    test_ = {}
    
    for id, race in train_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            #nicht gewollte Attribute werden aus Datensatz entfernt (bspw. podium_position)
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            #die letze runde eines Fahrers wird betrachtet (enthält kumulierte Informationen über zuvor gefahrene Runden)
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[cols].values)
            train.append((x_tensor, [temp_y[0]]))
        train_[id] = train
        train = []
    for id, race in dev_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            dev.append((x_tensor, [temp_y[0]]))
        dev_[id] = dev
        dev = []
    for id, race in test_data.items():
        for did in race.driverId.unique():
            temp = race.where(race.driverId == did).dropna(how = "all")
            temp_y = list(temp["podium_position"])
            cols = [col for col in temp.columns if col not in nogo_columns]
            temp_x = temp[cols]
            temp_x = temp_x.tail(1)
            x_tensor = torch.tensor(temp_x[temp_x.columns].values)
            test.append((x_tensor, [temp_y[0]]))
        test_[id] = test
        test = []
        
    return train_,dev_,test_

Einlesen der Dateien als Dataframes, welche in zwei Dictionaries gespeichert werden:<br>
- Attribut directory verweißt auf den Ordnernamen der sliced Dateien, Default ist 'sliced_data'
- Rennen sind standardmäßig nach 50% gesliced (siehe hierzu Datenaufbereitung)

<br> Einmal für das ganze Notebook wird festgelegt, ob Cuda beim Training der Neuronalen Netze verwendet werden soll:

In [3]:
cuda = input('Cuda? [y/n]: ')
sliced_races = load_data(directory = 'sliced_data')

Cuda? [y/n]: n
Einlesen der sliced Dateien erfolgreich


Aufteilen des Datensatzes in einen Trainings-, einen Development- und einen Testteil mit Hilfe der Funktion train_dev_test(). Standardmäßig wird der Datensatz zu 70% in Trainings-, zu 20% in Dev- und zu 10% in Testdaten aufgeteilt. Die Verteilung kann mit den Parametern train_p,dev_p,test_p die der Funktion train_dev_test() übergeben werden können, angepasst werden. Wichtig ist bloß, dass insgesamt train_p,dev_p und test_p 1 ergeben. Die Datenbeispiele aus sliced_races werden vor dem Aufteilen geshuffled. Die Funktion train_dev_test() gibt die Datenbeispiele dann wieder in Dictionary Form zurück, wobei jeder RaceId ein DataFrame mit Renndaten zugeordnet wird. Im nächsten Schritt werden die drei Datensammlungen der Funktion to_tensor() übergeben, die nicht gewünschte Spalten aus den Datensätzen entfernt (nogo_columns) und diese dann in Tensoren abspeichert, die wiederrum  ihrem Targetvalue (podiums_position) zugeordnet werden, um die Überprüfung der Vorhersagen später zu vereinfachen. Tupel mit Datensatz und Targetvalue werden dann wieder in dem jeweiligen Dictionary abgespeichert und ihrer RaceId zugeordnet.

In [5]:
#definieren nicht gewollter Attribute (Attribute die nicht dem Modell übergeben werden sollen):
nogo_columns = ['year', 
                'podium_position', 
                'raceId',
                'lap_number',
                'driverId', 
                'driver_fullname',
                'total_milliseconds',
               'lap_in_milliseconds',
               'total_minutes']
#train dev test splitting
train, dev, test = train_dev_test(sliced_races)
#umwandeln in Tensoren und entfernen der nogo_columns
train_T, dev_T, test_T = to_tensor(train, dev, test, nogo_columns)

print('Train:',len(train))
print('Dev:',len(dev))
print('Test:',len(test))
print('Rennen insgesamt:', len(train)+len(dev)+len(test))
print(25*'=')
print('Tensoren:')
print('Train:',len(train_T))
print('Dev:',len(dev_T))
print('Test:',len(test_T))
print('Rennen insgesamt:', len(train_T)+len(dev_T)+len(test_T))

Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137
Tensoren:
Train: 96
Dev: 27
Test: 14
Rennen insgesamt: 137


#### Definieren von notwendigen Klassen: 
    
Zuerst werden zwei Klassen für Neuronale Netze definiert. Die erste Klasse definiert ein statisches Neuronales Netz, von dem wir wissen, dass es auf unseren Daten gut funktioniert. Die zweite Klasse definiert ein Dynamisches Neuronales Netz, welches in der Hyperparamteroptimierung verwendet wird, um Netze zu erzeugen, die gegen das statische Netz getestet werden.
Der dynamischen Klasse werden Dictionarys übergeben, die spezifizieren wie das Neuronale Netz erzeugt werden soll. Ein Beispiel kann wie folgt aussehen:<br>

{
                 'first'    :    ['linear', 10,150],<br>
                 'relu0'    :    ['linear',150,180],<br>
                 'relu1'    :    ['dropout',0,0],<br>
                 'no_activation0'    :    ['linear',180,190],<br>
                 'relu2'    :    ['linear',190,120],<br>
                 'relu3'    :    ['linear',120,100],<br>
                 'relu4'    :    ['linear',100,70],<br>
                 'relu5'    :    ['linear',70,30],<br>
                 'relu6'    :    ['linear',30,1]
}<br>
         
Dieses Dictionary würde ein Netz erzeugen, wie es in der statischen Klasse definiert wird. Die Keys des Dictionarys geben die dem Layer vorhergehende Activationfunction an (keine für das erste Layer, aus diesem Grund das Stichwort first), während der zugeordnete Value eine Liste mit Layerspezifikationen enthält. An erster Stelle wird der Typ des Layers definiert (dropout oder linear), an zweiter Stelle die Anzahl der Input Neuronen und an dritter Stelle die Anzahl der Output Neuronen für das Layer.

Die nächsten Klassen werden für die Hyperparameteroptimierung definiert. In der ersten Klasse werden dynamisch Neuronale Netze erzeugt, die mit unterschiedlichen Layeranzahlen und Neuronenanzahlen erzeugt werden. Von jeder Layeranzahl wird eine Variation verschiedener Netze erzeugt, sodass eine feste Anzahl Netze mit einer festen Layeranzahl gegen Netz mit einer anderen Anzahl von Layern getestet werden kann. <br>
In der zweiten Klasse werden die Parameter Trainigsepochen und Lernrate für den Adam OPtimizer optimiert. Die Klasse erhält das statische Neuronale Netz und das beste der dynamisch erzeugten Neuronalen Netze und optimiert diese Parameter für beide Netze.

In [6]:
class Netz(nn.Module):
    def __init__(self):
        super(Netz,self).__init__()
        #self.fc1 = nn.Linear(53, 150)
        self.fc1 = nn.Linear(52, 150)
        self.fc2 = nn.Linear(150, 180)
        self.fc3 = nn.Linear(180, 190)
        self.fc4 = nn.Linear(190, 120)
        self.fc5 = nn.Linear(120, 100)
        self.fc6 = nn.Linear(100, 70)
        self.fc7 = nn.Linear(70, 30)
        self.fc8 = nn.Linear(30, 1)
        self.dropout = nn.Dropout()
        
        
    def forward(self,x):
        x = self.fc1(x.float())
        x = F.relu(x.float())
        x = self.fc2(x.float())
        x = F.relu(x.float())
        x = self.dropout(x)
        x = self.fc3(x.float())
        x = F.relu(x.float())
        x = self.fc4(x.float())
        x = F.relu(x.float())
        x = self.fc5(x.float())
        x = F.relu(x.float())
        x = self.fc6(x.float())
        x = F.relu(x.float())
        x = self.fc7(x.float())
        x = F.relu(x.float())
        x = self.fc8(x.float())
        return x

class NetzDynamic(nn.Module):
    
    def __init__(self, layer_information):
        super(NetzDynamic,self).__init__()
        self.__layer_information = layer_information
        #definieren von einer Moduleliste, die alle Layer des Netzes enthalten wird
        self.layers = nn.ModuleList()
        
        #erzeugen der in layer_information übergebeben Layer und hinzufügen zu layers Moduleliste
        for specs in self.__layer_information.values():
            type_ = specs[0]
            in_ = specs[1]
            out_ = specs[2]
            if type_ == 'linear':
                self.layers.append(nn.Linear(in_,out_))
            if type_ == 'dropout':
                self.layers.append(nn.Dropout())
        
    def forward(self, x):
        #definieren eines dynamischen forward pass, activation function richtet sich nach dictionary key
        lay_idx = 0
        for activation, specs in self.__layer_information.items():
            if activation.startswith('relu'):
                layer = self.layers[lay_idx] #auswählen des relevanten Layers aus layers ModuleList
                if specs[0] == 'dropout':
                    x = F.relu(x.float())#anwenden der activation function
                    x = layer(x)
                else:
                    x = F.relu(x.float())
                    x = layer(x.float()) 
            if activation.startswith('sigmoid'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.sigmoid(x.float())
                    x = layer(x)
                else:
                    x = torch.sigmoid(x.float())
                    x = layer(x.float())
                    
            if activation.startswith('tanh'):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = torch.tanh(x.float())
                    x = layer(x)
                else:
                    x = torch.tanh(x.float())
                    x = layer(x.float())
            if activation.startswith("first"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float())
            if activation.startswith("no_activation"):
                layer = self.layers[lay_idx]
                if specs[0] == 'dropout':
                    x = layer(x)
                else:
                    x = layer(x.float()) 
            lay_idx += 1#updaten der layerid, um forward pass ins nächste layer zu machen
        
        return x

In [15]:
class HP_Layer_Optimizer(object):
    
    def __init__ (self, 
                  layer_range = (2,8), #range mindestanzahl layer, max anzahl layer
                  input_start = 52, #anzahl inputneuronen für das Inputlayer
                  output_last = 1, #anzahl outputneuronen für das letzte layer
                  in_out_range = (30,200), #range in der sich die anzahl der Neuronen für die Hiddenlayer bewegen soll
                  random_activation = False,#boolean, die angibt ob activation functions random gepicked werden sollen
                  types = ['linear', 'dropout'],#layertypen, von denen gewählt wird
                  activations = ['relu', 'sigmoid', 'tanh'],#activation functions
                  activation_dist = [0.5,0.25,0.25],#wenn random_activation == False wird hier die Häufigkeit für jede Activation übergeben, nicht implementiert
                  dropout_num = 1,#anzahl von dropoutlayern pro NN
                  cuda = True,#soll mit cuda gearbeitet werden?
                  lr = 0.0001,#lernrate für adam optimizer
                  max_epochs = 5,#anzahl der Trainingsepochen
                  create_combinations = True, #sollen von der Klasse Netze erzeugt werden, oder sollen diese von außen in die Klasse gegeben werden?
                  create_variations = False,#soll von jeder Layeranzahl mehr als ein Netz erzeugt werden?
                  num_variations = 4, #wenn Variationen erzeugt werden sollen wird eine festgelegte Zahl Netze mit 4 Layer, eine festgelegte Anzahl mit 5 Layern usw. erzeugt
                  pure_activations = True #wenn random_activation auf True gesetzt ist kann hier festgelegt werden, dass zusätzlich zu absolut zufällig kombinierten activationfunctions
                  #auch noch Netze erzeugt werden, die nur einheitliche activation functions haben (pure sind)
                  ):
        
        self.__model = None #model, welches trainiert und getestet wird, wird hier zwischengespeichert
        self.__layer_range = layer_range #min und max anzahl von layern in Tupelform
        self.__input_start = input_start#Inputgröße für Startlayer/Inputlayer
        self.__output_last = output_last#Outputgröße für letztes Layer / Outputlayer
        self.__in_out_range = in_out_range#range in der sich die Anzahl der In- und Outputs für die Hiddenlayer bewegen soll (Tupelform)
        self.__layer_types = types#liste welche Layertypen enthält (bspw. linear oder dropout)
        self.__dropout_number = dropout_num#anzahl von layern, die pro Netz ein dropout Layer sein sollen
        self.__random_activation = random_activation#Boolean, ob die activation zufällig gewählt werden soll
        self.__create_variations = create_variations
        self.__number_of_variations = num_variations
        self.__pure = pure_activations
        
        self.__combination_results = {}#dictionary, die dem schlüssel zu einer NN Kombi einen MAE zuordnet
        self.model_specs_combinations = {}#dictionary, welches die jeweiligen NN Kombinationen enthält (einem Schlüssel zugeordnet)
        self.train_data = None#trainingsdaten, die dem Optimizer übergeben werden für die Modelle
        self.test_data = None#developmentdaten, um die Modelle zu testen und Aussagen über die besten Kombinationen zu treffen
        self.lr = lr#lernrate für Modell OPtimizer (default Adam)
        self.max_epochs = max_epochs#Anzahl der Trainingsepochen
        self.cuda = cuda#Boolean, ob mit cuda gearbeitet werden soll oder nicht
        self.opt_combination = {}#dictionary enthält optimale kombination aus möglichen kombinationen
        
        self.__activations = None#entweder Liste (wenn activations random ausgewählt werden sollen), oder dictionary, 
        #wenn activation functions mit einer bestimmten Häufigkeit verwendet werden sollen
        if self.__random_activation:#random pick von activation functions
            self.__activations = activations
        else:#activation function sollen mit einer gewissen häufigkeit ausgewählt werden
            k = {}
            for a in range(len(types)):#zuordnen einer häufihkeit aus activation_dist liste zu jeder activation function aus types
                act = types[a]
                dist = activation_dist[a]
                k[act] = dist
            self.__activations = k
            
        #aufrufen der Funktion, die anhand der übergebenen Parameter Modellkombinationen erzeugt
        if create_combinations:
            self.__create_combinations()
        else:
            print('NN Kombinationen müssen in Dictionary Form selbst übergeben werden')
        
    def __train(self, epoch, optimizer):
        '''
            funktion übernimmt das Training von dem in self.__model
            zwischengespeicherten NN. 
            Epoch: Jetzige Epoche in der trainiert wird (für coolen print Befehl wichtig)
            Optimizer: Optimizer mit dem Parameter des NN aus self.__model optimiert werden
        '''
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)
                    optimizer.zero_grad()
                    out = self.__model(data)
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
            
    def __test(self):
        '''
            Funktion, die das Testen des Models aus self.__model auf den übergebenen Dev-Daten 
            (self.test_data) übernimmt und den gesamt MAE für das jeweilige Modell berechnet
        '''
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    out = out.detach().numpy()
                    target = target.cpu()
                    target = target.detach().numpy()
                    total += abs(out - target[0][0])
                    help_dict[target[0][0]] = out
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)
                    out = self.__model(data)
                    out = out.detach().numpy()
                    target = target.detach().numpy()
                    total += abs(out - target[0][0])
                    help_dict[target[0][0]] = out
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
    
    def get_all_information(self):
        
        print('All Model Combinations with encoding:\n', self.model_specs_combinations)
        print('Model Results:\n', self.__combination_results)
        print('Optimale Kombination:\n', self.opt_combination)
        
    def validate_combinations(self):
        
        for key, combination in self.model_specs_combinations.items():
            print(combination)
            self.__model = NetzDynamic(combination)
            optimizer = optim.Adam(self.__model.parameters(), lr = self.lr)
            #trainieren des modells
            for epoch in range(1,self.max_epochs):
                self.__train(epoch, optimizer)  
            #Aufrufen der Testfunktion
            result = self.__test()
            A = result.prediction.tolist()
            y = result.target.tolist()
            mae = MAE(A,y)
            
            
            
            self.__combination_results[key] = mae
            
        #finden der besten kombination nach minimalstem Error (MAE)
        key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
        best_combination = self.model_specs_combinations[key_min]
        best_combination['mae'] = self.__combination_results[key_min]
        self.opt_combination = best_combination
        #self.__combination_overview[key] = specifics
        
        
    def __create_combinations(self):
        '''
            Funktion erzeugt Dictionarys, mit NN Modellspezifikationen, die
            später gegeneinander getestet werden sollen. Kombinationen werden
            in dem Dictionary self.model_specs_combinations unter einem 
            Schlüssel abgespeichert
        '''
        
        if self.__create_variations:
            
            if self.__pure:
                '''
                es werden mehr als ein Netz von einer bestimmten Layeranzahl erzeugt:
                wenn die Layeranzahl Range zwischen 7 und 9 liegt, self.__create_variations = True ist und
                die Variable self.__number_of_variations = 4 ist werden 4 Netze mit 7 Layern und 4 Netze
                mit 8 Layern erzeugt. Diese können dann gegeneinander getestet werden und geben eine bessere
                Übersicht über eine gute Layeranzahl. Wenn self.__pure auf True gesetzt wird, wird aus der 
                Liste mit activationfunctions [relu, sigmoid, tanh] als default 'reine' Netze erzeugt, die immmer
                nur eine der Funktionen enthält (Relu als anfang). Danach werden diese netze kopiert, nur dass die 
                Aktivierungsfunktionen überschrieben werden (ReLu wird durch sigmoid und tanh ersetzt). Zuletzt werden 
                die Funktionen wild gemischt.
                '''
                min_layer = self.__layer_range[0]
                max_layer = self.__layer_range[1]
                if min_layer == max_layer:
                    max_layer += 1
                #pure ist true, deswegen wird zu anfang eine der aktivierungsfunktionen ausgewählt, die fürs ganze netz gewählt wird
                act = random.choice(self.__activations)
                for layer in range(min_layer, max_layer): 
                    variation_counter = 0
                    while variation_counter < self.__number_of_variations:
                        variation_counter +=1 
                        dropout_counter = 0
                        act_count = 0
                        specs_dict = {}
                        middle = layer//2
                        for l in range(layer):
                            layer_specs = []
                            key = act+str(act_count)
                            act_count += 1
                            if dropout_counter == self.__dropout_number:
                                #es wurden schon ausreichend dropout layer erzeugt
                                l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                            else:
                                l_ = random.choice(self.__layer_types)
                                if l_ == 'dropout':
                                    dropout_counter +=1
                                    
                            if l == 0:
                                in_ = self.__input_start
                                if l_ == 'dropout':
                                    dropout_counter = dropout_counter-1
                                    l_ = 'linear'
                                range_start = self.__in_out_range[0]
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            else:
                                layer_before = specs_dict[list(specs_dict.keys())[-1]]
                                out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                                l_type = layer_before[0]
                                i_ltype = -2
                                while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                    #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                    layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                    l_type = layer_before[0]
                                    out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                    i_ltype = i_ltype -1
                                if l <= middle: #in der ertsen hälfte nimmt output zu
                                    in_ = out_alt
                                    range_start = out_alt
                                    range_end = self.__in_out_range[1]
                                    out_ = random.randint(range_start, range_end)
                                else:#in zweiter hälfte der layer nimmt output wieder ab
                                    in_ = out_alt
                                    range_start = self.__in_out_range[0]
                                    range_end = out_alt
                                    out_ = random.randint(range_start, range_end)
                                layer_specs.append(l_)
                                layer_specs.append(in_)
                                layer_specs.append(out_)
                            if l == layer-1:
                                l_ = 'linear'
                                layer_specs = [l_,in_,self.__output_last]
                                specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                                
                            elif l == 0:
                                specs_dict['first'] = layer_specs
                            else:
                                if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                    key = 'no_activation'+str(act_count)
                                specs_dict[key] = layer_specs
                        key = random.randint(0,10000)
                        while key in list(self.model_specs_combinations.keys()):
                             key = random.randint(0,10000)
                        self.model_specs_combinations[key]= specs_dict   
                        print(specs_dict,'\n')
                used_acts = [act]
                act_count = 0
                netze = list(self.model_specs_combinations.values())
                for activation in self.__activations:
                    if activation in used_acts:
                        continue
                    else:
                        for netz in netze:#alle bisher erzeugten netze werden betrachtet
                            key = random.randint(0,10000)
                            while key in list(self.model_specs_combinations.keys()):
                                key = random.randint(0,10000)
                            netz_neu = {}
                            for act_alt, layer in netz.items():
                                if act_alt == 'no_activation' or act_alt == 'first':
                                    netz_neu[act_alt] = layer
                                else:
                                    act_ = activation+str(act_count)
                                    act_count += 1
                                    netz_neu[act_] = layer
                            print(netz_neu,'\n')
                            self.model_specs_combinations[key] = netz_neu
            else:
                raise Exception('please et pure_activations to True, False not implemented')
                    
        else:
            min_layer = self.__layer_range[0]
            max_layer = self.__layer_range[1]
            if min_layer == max_layer:
                max_layer += 1
            for layer in range(min_layer, max_layer): 
                
                dropout_counter = 0
                act_count = 0
                specs_dict = {}
                middle = layer//2
                for l in range(layer):
                    layer_specs = []
                    if self.__random_activation:#random activation pick ist aktiviert
                        act = random.choice(self.__activations)
                        key = act+str(act_count)
                        act_count += 1
                        if dropout_counter == self.__dropout_number:
                            #es wurden schon ausreichend dropout layer erzeugt
                            l_ = random.choice([x for x in self.__layer_types if x not in ['dropout']])
                        else:
                            l_ = random.choice(self.__layer_types)
                            if l_ == 'dropout':
                                dropout_counter +=1
                                
                        if l == 0:
                            in_ = self.__input_start
                            if l_ == 'dropout':
                                dropout_counter = dropout_counter-1
                                l_ = 'linear'
                            range_start = self.__in_out_range[0]
                            range_end = self.__in_out_range[1]
                            out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        else:
                            layer_before = specs_dict[list(specs_dict.keys())[-1]]
                            out_alt = specs_dict[list(specs_dict.keys())[-1]][2]
                            l_type = layer_before[0]
                            i_ltype = -2
                            while l_type == 'dropout':#überprüfen ob vorhergegangenes Layer ein dropout layer war
                                #sobald das vorhergegangene nicht-dropoutlayer gefunden wurde, wird output größe übernommen
                                layer_before = specs_dict[list(specs_dict.keys())[i_ltype]]
                                l_type = layer_before[0]
                                out_alt = specs_dict[list(specs_dict.keys())[i_ltype]][2]
                                i_ltype = i_ltype -1
                            if l <= middle: #in der ertsen hälfte nimmt output zu
                                in_ = out_alt
                                range_start = out_alt
                                range_end = self.__in_out_range[1]
                                out_ = random.randint(range_start, range_end)
                            else:#in zweiter hälfte der layer nimmt output wieder ab
                                in_ = out_alt
                                range_start = self.__in_out_range[0]
                                range_end = out_alt
                                out_ = random.randint(range_start, range_end)
                            layer_specs.append(l_)
                            layer_specs.append(in_)
                            layer_specs.append(out_)
                        #if l == layer-1:
                        #    l_ = 'linear'
                        #    layer_specs = [l_,in_,self.__output_last]
                        #    specs_dict['last'] = layer_specs#layer wird ohne activation gespeichert und als letztes Layer des NN gekennzeichnet
                            
                        #elif l == 0:
                        if l == 0:
                            specs_dict['first'] = layer_specs
                        else:
                            #if l_ == 'dropout':#keine activation function bei dropout layern
                            #    key = 'no_activation'+str(act_count)
                            if specs_dict[list(specs_dict.keys())[-1]][0] == 'dropout':# keine activation function bei layern direkt nach einem dropout layer
                                key = 'no_activation'+str(act_count)
                            specs_dict[key] = layer_specs
                key = random.randint(0,10000)
                while key in list(self.model_specs_combinations.keys()):
                     key = random.randint(0,10000)
                self.model_specs_combinations[key]= specs_dict   
                print(specs_dict)

                
class HP_Optimizer(object):
    
    def __init__(self, lr_range = (0.0001,0.0001), step_size = 0.0001, max_epochs = (2,2), opt = 'Adam', cuda = True, dynamic = False, dyn_combination = {}):
        
        self.__model = Netz()
        self.__lr = lr_range
        self.__epochs = max_epochs
        self.__optimizer = opt
        self.__steps = step_size
        self.__combination_results = {}
        self.__combination_overview = {}
        self.train_data = None
        self.test_data = None
        self.cuda = cuda
        self.opt_combination = {}
        self.__dynamic = dynamic
        self.__dyn_combination = dyn_combination
        
        
    def validate_combinations(self):
        
        specifics = {}
        if self.__optimizer == 'Adam':
            
            #definieren der range für die lernratenoptimierung
            '''
            
            IST HIER WAS KAPUTT?
            
            '''
            if isinstance(self.__lr, tuple):
                lr_s = self.__lr[0]
                lr_e = self.__lr[1]
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if lr_s == lr_e:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = lr_s
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in np.arange(lr_s,lr_e, self.__steps):
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if lr_s == lr_e:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = lr_s)
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = lr_s
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in np.arange(lr_s,lr_e, self.__steps):
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
            elif isinstance(self.__lr, list):
                #wurde eine range für die anzahl der epochen übergeben?
                if self.__epochs[0] == self.__epochs[1]:
                    #hyperparamter epochenanzahl wird nicht optimiert
                    max_epoch = self.__epochs[0]
                    if len(self.__lr) ==1:
                        #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                        print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                        #setzen des optimizers als Adam und erzeugen des Modells
                        if self.__dynamic:
                            self.__model = NetzDynamic(self.__dyn_combination)
                        else:
                            self.__model = Netz()
                        optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                        for epoch in range(1,max_epoch):
                            self.__train(epoch, optimizer)     
                        result = self.__test()
                        A = result.prediction.tolist()
                        y = result.target.tolist()
                        mae = MAE(A,y)
                        specifics = {}
                        specifics['lr'] = self.__lr[0]
                        specifics['epochen'] = max_epoch
                        key = random.randint(0,10000)
                        while key in list(self.__combination_results.keys()):
                             key = random.randint(0,10000)
                        self.__combination_results[key] = mae
                        self.__combination_overview[key] = specifics 
                    else:
                        for l in self.__lr:
                            #setzen des optimizers als Adam und erzeugen des Modells
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = l)
                            #trainieren des modells
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)  
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = l
                            specifics['epochen'] = max_epoch
                            #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics
                else:
                    for max_epoch in range(self.__epochs[0], self.__epochs[1]):
                        #definieren der range für die lernratenoptimierung
                        lr_s = self.__lr[0]
                        lr_e = self.__lr[1]
                        #hyperparamter epochenanzahl wird nicht optimiert
                        if len(self.__lr) == 1:
                            #es ist keine range für die lernrate gegeben, in der diese optimiert werden soll
                            #print('Parameter Epochen und Lernrate können nicht optimiert werden, da kein Intervall übergeben wurde')
                            if self.__dynamic:
                                self.__model = NetzDynamic(self.__dyn_combination)
                            else:
                                self.__model = Netz()
                            optimizer = optim.Adam(self.__model.parameters(), lr = self.__lr[0])
                            for epoch in range(1,max_epoch):
                                self.__train(epoch, optimizer)       
                            result = self.__test()
                            A = result.prediction.tolist()
                            y = result.target.tolist()
                            mae = MAE(A,y)
                            specifics = {}
                            specifics['lr'] = self.__lr[0]
                            specifics['epochen'] = max_epoch
                            key = random.randint(0,10000)
                            while key in list(self.__combination_results.keys()):
                                 key = random.randint(0,10000)  
                            self.__combination_results[key] = mae
                            self.__combination_overview[key] = specifics          
                        else:
                            for l in self.__lr:
                                if self.__dynamic:
                                    self.__model = NetzDynamic(self.__dyn_combination)
                                else:
                                    self.__model = Netz()
                                optimizer = optim.Adam(self.__model.parameters(), lr = l)
                                #trainieren des modells
                                for epoch in range(1,max_epoch):
                                    self.__train(epoch, optimizer)  
                                result = self.__test()
                                A = result.prediction.tolist()
                                y = result.target.tolist()
                                mae = MAE(A,y)
                                specifics = {}
                                specifics['lr'] = l
                                specifics['epochen'] = max_epoch
                                #abspeichern der gewonnenen informationen (MAE nach lr und anzahl durchgeführter epochen)
                                key = random.randint(0,10000)
                                while key in list(self.__combination_results.keys()):
                                     key = random.randint(0,10000)
                                self.__combination_results[key] = mae
                                self.__combination_overview[key] = specifics
                #finden der besten kombination nach minimalstem Error (MAE)
                key_min = min(self.__combination_results.keys(), key=(lambda k: self.__combination_results[k]))
                best_combination = self.__combination_overview[key_min]
                best_combination['mae'] = self.__combination_results[key_min]
                self.opt_combination = best_combination
                
        else:
            raise ('No valid optimizer given! Try Adam for example!')
            
    def __train(self, epoch, optimizer):
        if self.cuda:
            self.__model.cuda()
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0).cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                        epoch, batch_id *len(data), len(self.train_data),
                    100. * batch_id / len(self.train_data), loss.item()))
                    batch_id +=1
        else:
            self.__model.train()
            batch_id = 0
            for key in self.train_data.keys():
                for data, target in self.train_data[key]:
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)#.cuda()
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    optimizer.zero_grad()
                    out = self.__model(data)
                    #print("Out: ", out, out.size())
                    #print("Target: ", target, target.size())
                    criterion = nn.MSELoss()
                    loss = criterion(out, target)
                    loss.backward()
                    optimizer.step()
                    #print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    #    epoch, batch_id *len(data), len(train_data),
                    #100. * batch_id / len(train_data), loss.item()))
                    batch_id +=1
                    
            
    def __test(self):
        total = 0
        count = 0
        result_dict = {}
        result = pd.DataFrame(columns = ['target','prediction'])
        help_dict = {}
        if self.cuda:
            self.__model.cuda()
            for key in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[key]:
                    self.__model.eval()
                    #files.listdir(path)
                    data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1).cuda()
                    out = self.__model(data).cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                
                result = result.append(t, sort = True)
                
        else:
            for raceId in self.test_data.keys():
                #print(key)
                help_dict = {}
                for data, target in self.test_data[raceId]:
                    self.__model.eval()
                    #files.listdir(path)
                    #data = data.cuda()
                    target = torch.Tensor(target).unsqueeze(0)
                    shape = target.size()[1]
                    target = target.resize(shape,1)#.cuda()
                    out = self.__model(data)#.cpu()
                    #print(out)
                    out = out.detach().numpy()
                    #out = np.round(out)
                    #target = target.cpu()
                    target = target.detach().numpy()
                    #print(data)
                    #print(data["driverId"])
                    total += abs(out - target[0][0])
                    #print("current_position: ", data[0][0].item())
                    #print("Output: ", out)
                    #print("Target: ", target)
                    help_dict[target[0][0]] = out
                    #print("Difference: ", out - target)
                    count+=1
                #Auslesen der predicteten Werte A und der zugehörigen targets y
                A = [x[0][0] for x in list((help_dict.values()))]
                y = list(help_dict.keys())
                
                #Anfügen der Werte an Result
                t = pd.DataFrame()
                t['target'] = y
                t['prediction_value'] = A
                t = sqldf.sqldf('''select * from t order by prediction_value ASC''')
                t.reset_index(inplace = True)
                t.rename(columns = {'index':'prediction'}, inplace = True)
                t['prediction'] = t['prediction']+1
                
                result = result.append(t, sort = True)
        return result
         
            
    def get_all_information(self):
        
        print('Chosen Model:',self.__model)
        print('Learningrate Range:',self.__lr)
        print('Maximum Epochs:', self.__epochs)
        print('Chosen Optimizer:', self.__optimizer)
        print('Result Encoding:', self.__combination_overview)
        print('Results:', self.__combination_results)
        print('Optimale Kombination:', self.opt_combination)
        
    def help(self):
        print('Parameters with defaults:\nlr_range --> (0.0001,0.0001),\nstep_size--> 0.0001,\nmax_epochs-->(2,2),\nopt-->"Adam",\ncuda=True')
        print('lr_range: Tupel with learnrate range')
        print('step_size: float/int for step_size of learnrate')
        print('max_epochs: Tupel with number of epochs range')
        print('opt: Optimizer (by default Adam)')
        print('cuda: True/False if cuda should be used, default = True\n')
        print('Attributes:')
        print('set self.train_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('set self.test_data as dictionary with races (form: {raceId: race(dataframe)})')
        print('self.opt_combination: Dictionary which contains the best combination of the given parameters\n')
        print('Methods:')
        print('call self.validate_combination() to compare all combinations')
        print('get all information/results with self.get_all_information()')

#### Finden des "optimalsten" Netzes  
Aufrufen des Layeroptimizers, um verschiedene Neuronale Netze gegeneinander zu testen, mit verschiedenen Layeranzahlen, und Input und Output Neuronen. Gemeinsam haben die Netze, dass bis zur Mitte die Anzahl Neuronen zunimmt, und dann zum letzten Layer hin wieder abnimmt.
<br>Es wird das Netz ausgewählt, welches den niedrigsten MAE auf dem Dev Datensatz hat. Ziel ist es durch Aufrufen dieser Klasse die beste Layeranzahl-Aktivierungsfunktionen-Kombination für den Use Case zu finden (anhand vorgegebener einschränkender Paramter). Da die Anzahl der Neuronen jedes Mal zufällig implementiert wird, und dieser Faktor nur wenig eingeschränkt ist, kann hier mit jedem neuen Aufruf der Klasse ein anderes Netz als optimal ausgewählt werden. Das optimalste Netz richtet sich auch stark nach dem train-dev-test splitting des Datensatzes.

In [16]:
if cuda.lower() == 'y':
    l = HP_Layer_Optimizer(layer_range=(7,12),
                       max_epochs = 6,
                       input_start = 52,
                       #activations = ['relu'], 
                       random_activation=True,
                       create_combinations=True, 
                       create_variations = True, 
                       num_variations = 5,
                       pure_activations = True,
                       cuda = True)
else:
    l = HP_Layer_Optimizer(layer_range=(10,12),
                       max_epochs = 4,
                       input_start = 52,
                       #activations = ['relu'], 
                       random_activation=True,
                       create_combinations=True, 
                       create_variations = True, 
                       num_variations = 5,
                       pure_activations = True,
                       cuda = False)
#l.model_specs_combinations = netze
l.train_data = train_T
l.test_data = dev_T
#l.validate_combinations()
#l.get_all_information()
opt_combination = l.opt_combination

{'first': ['linear', 52, 196], 'sigmoid1': ['dropout', 196, 198], 'no_activation3': ['linear', 196, 200], 'sigmoid3': ['linear', 200, 200], 'sigmoid4': ['linear', 200, 200], 'sigmoid5': ['linear', 200, 200], 'sigmoid6': ['linear', 200, 149], 'sigmoid7': ['linear', 149, 146], 'sigmoid8': ['linear', 146, 62], 'last': ['linear', 62, 1]} 

{'first': ['linear', 52, 59], 'sigmoid1': ['dropout', 59, 73], 'no_activation3': ['linear', 59, 141], 'sigmoid3': ['linear', 141, 157], 'sigmoid4': ['linear', 157, 180], 'sigmoid5': ['linear', 180, 194], 'sigmoid6': ['linear', 194, 81], 'sigmoid7': ['linear', 81, 62], 'sigmoid8': ['linear', 62, 35], 'last': ['linear', 35, 1]} 

{'first': ['linear', 52, 34], 'sigmoid1': ['dropout', 34, 85], 'no_activation3': ['linear', 34, 148], 'sigmoid3': ['linear', 148, 170], 'sigmoid4': ['linear', 170, 175], 'sigmoid5': ['linear', 175, 187], 'sigmoid6': ['linear', 187, 49], 'sigmoid7': ['linear', 49, 36], 'sigmoid8': ['linear', 36, 35], 'last': ['linear', 35, 1]} 

{'

#### Optimieren der Lernrate und Trainingsepochenanzahl
Paralleles Aufrufen des Optimierers, übergeben der zuvor erzeugten Dev- und Trainingsdatensätze, angeben der Intervalle für die die Hyperparameter getestet werden sollen (Lernrate und Epochenanzahl). Für das dynamische Modell wird das Dictionary übergeben, welches von dem vorhergegangenen Layer Optimizer am besten bewertet wurde und die Variable dynamic auf True gesetzt (default False). Der "normale" HP_Optimizer basiert auf der Klasse Netz().

In [8]:
opt_={}
for key in opt_combination.keys():
    if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
        opt_[key] = opt_combination[key]
#überprüfen ob cuda verwendet werden soll, dann Aufruf des Optimizers für beide Netze
if cuda.lower() == 'y':
    h = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = True)
    h_dynamic = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = True, dynamic = True, dyn_combination = opt_)
else:
    h = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False)
    h_dynamic = HP_Optimizer(lr_range = [0.0001,0.00045],step_size = 0.0001, max_epochs=(3,5),cuda = False, dynamic = True, dyn_combination = opt_)

h.help()
#Zuweisen der Trainingsdaten
h.train_data = train_T
h_dynamic.train_data = train_T
#Zuweisen der Dev Daten als Testdaten
h.test_data = dev_T
h_dynamic.test_data = dev_T
#Aufruf der Funktion die verschiedene Kombinationen der Epochen und Lernraten gegeneinander vergleicht
h.validate_combinations()
h_dynamic.validate_combinations()
#Ausgabe der Ergebnisse des Vergleichs aus self.validate_combinations()
print('Statisch Netz():\n')
h.get_all_information()
print('\n',25*'=','\n')
print('Dynamisches Netz:\n')
h_dynamic.get_all_information()

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Parameters with defaults:
lr_range --> (0.0001,0.0001),
step_size--> 0.0001,
max_epochs-->(2,2),
opt-->"Adam",
cuda=True
lr_range: Tupel with learnrate range
step_size: float/int for step_size of learnrate
max_epochs: Tupel with number of epochs range
opt: Optimizer (by default Adam)
cuda: True/False if cuda should be used, default = True

Attributes:
set self.train_data as dictionary with races (form: {raceId: race(dataframe)})
set self.test_data as dictionary with races (form: {raceId: race(dataframe)})
self.opt_combination: Dictionary which contains the best combination of the given parameters

Methods:
call self.validate_combination() to compare all combinations
get all information/results with self.get_all_information()


c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
c:\users\hwebe\appdata\local\programs\python\

Statisch Netz():

Chosen Model: Netz(
  (fc1): Linear(in_features=52, out_features=150, bias=True)
  (fc2): Linear(in_features=150, out_features=180, bias=True)
  (fc3): Linear(in_features=180, out_features=190, bias=True)
  (fc4): Linear(in_features=190, out_features=120, bias=True)
  (fc5): Linear(in_features=120, out_features=100, bias=True)
  (fc6): Linear(in_features=100, out_features=70, bias=True)
  (fc7): Linear(in_features=70, out_features=30, bias=True)
  (fc8): Linear(in_features=30, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
Learningrate Range: [0.0001, 0.00045]
Maximum Epochs: (3, 5)
Chosen Optimizer: Adam
Result Encoding: {1023: {'lr': 0.0001, 'epochen': 3}, 3294: {'lr': 0.00045, 'epochen': 3}, 5810: {'lr': 0.0001, 'epochen': 4}, 919: {'lr': 0.00045, 'epochen': 4, 'mae': 1.3952569169960474}}
Results: {1023: 1.5612648221343874, 3294: 1.4505928853754941, 5810: 1.541501976284585, 919: 1.3952569169960474}
Optimale Kombination: {'lr': 0.00045, 'epo

c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


#### Testen der erzeugten Netze
Es werden die als optimal in den gegebenen Intervallen bestimmten Werte aus dem Optimierer genommen und zwei neue Netze mit ihren jeweiligen Parametern für Trainingsepochen und Lernrate trainiert (dies geschieht für das statische und das dynamische Netz). Zum Schluss werden beide Netze auf den Testdaten getestet und die Ergebnisse des Testlaufes ausgegeben.

In [9]:
max_epochs = h.opt_combination['epochen']
max_epochs_dyn = h_dynamic.opt_combination['epochen']
lr = h.opt_combination['lr']
lr_dynamic = h_dynamic.opt_combination['lr']
model = Netz()

opt={}
for key in opt_combination.keys():
    if key != 'mae':#mae wird noch in dictionary von HP_Optimizer hinzugefügt und muss entfernt werden
        opt[key] = opt_combination[key]
model_dynamic = NetzDynamic(opt)
if cuda.lower() == 'y':
    model.cuda() 
    model_dynamic.cuda()

    
optimizer = optim.Adam(model.parameters(), lr = lr)     
optimizer_d = optim.Adam(model_dynamic.parameters(), lr = lr_dynamic)   

def train_cuda(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer.zero_grad()
            out = model(data)
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
def train_(epoch):
    model.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            #data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)#.cuda()
            shape = target.size()[1]
            target = target.resize(shape,1)#.cuda()
            optimizer.zero_grad()
            out = model(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
            
def train_cuda_dynamic(epoch):
    model_dynamic.train()
    batch_id = 0
    for key in train_T.keys():
        
        for data, target in train_T[key]:
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0).cuda()
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            optimizer_d.zero_grad()
            out = model_dynamic(data)
            #print("Out: ", out, out.size())
            #print("Target: ", target, target.size())
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer_d.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
def train_dynamic(epoch):
    model_dynamic.train()
    batch_id = 0
    for key in train_T.keys():
        for data, target in train_T[key]:
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            optimizer_d.zero_grad()
            out = model_dynamic(data)
            criterion = nn.MSELoss()
            loss = criterion(out, target)
            loss.backward()
            optimizer_d.step()
            print("Train Epoche: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                epoch, batch_id *len(data), len(train),
            100. * batch_id / len(train), loss.item()))
            batch_id +=1
if cuda.lower() == 'y':
    for epoch in range(1,max_epochs):
        train_cuda(epoch) 
    for epoch in range(1,max_epochs_dyn):
        train_cuda_dynamic(epoch) 
else:
    for epoch in range(1,max_epochs):
        train_(epoch)  
    for epoch in range(1,max_epochs_dyn):
        train_dynamic(epoch)  

        
def test_times_cuda(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model(data).cpu()
            out = out.detach().numpy()
            target = target.cpu()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model.eval()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            out = model(data)
            out = out.detach().numpy()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times_cuda_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        help_dict = {}
        for data, target in test_final[key]:
            model_dynamic.eval()
            data = data.cuda()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1).cuda()
            out = model_dynamic(data).cpu()
            out = out.detach().numpy()
            target = target.cpu()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict

def test_times_dynamic(test_final):
    total = 0
    count = 0
    result_dict = {}
    help_dict = {}
    for key in test_final.keys():
        #print(key)
        help_dict = {}
        for data, target in test_final[key]:
            model_dynamic.eval()
            target = torch.Tensor(target).unsqueeze(0)
            shape = target.size()[1]
            target = target.resize(shape,1)
            out = model_dynamic(data)
            out = out.detach().numpy()
            target = target.detach().numpy()
            total += abs(out - target[0][0])
            help_dict[target[0][0]] = out
            count+=1
        result_dict[key] = help_dict
        
    return result_dict


if cuda.lower() =='y':
    total_results = test_times_cuda(test_T)
    total_results_dynamic = test_times_cuda_dynamic(test_T)
else:
    total_results = test_times(test_T)
    total_results_dynamic = test_times_dynamic(test_T)


c:\users\hwebe\appdata\local\programs\python\python37\lib\site-packages\torch\tensor.py:362: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


Train Epoche: 1 [0/96 (0%)]	Loss: 77.396187
Train Epoche: 1 [1/96 (1%)]	Loss: 33.503105
Train Epoche: 1 [2/96 (2%)]	Loss: 3.172037
Train Epoche: 1 [3/96 (3%)]	Loss: 116.235863
Train Epoche: 1 [4/96 (4%)]	Loss: 14.155195
Train Epoche: 1 [5/96 (5%)]	Loss: 248.172287
Train Epoche: 1 [6/96 (6%)]	Loss: 7.473963
Train Epoche: 1 [7/96 (7%)]	Loss: 94.756645
Train Epoche: 1 [8/96 (8%)]	Loss: 0.512228
Train Epoche: 1 [9/96 (9%)]	Loss: 59.218105
Train Epoche: 1 [10/96 (10%)]	Loss: 21.826233
Train Epoche: 1 [11/96 (11%)]	Loss: 186.735977
Train Epoche: 1 [12/96 (12%)]	Loss: 134.874039
Train Epoche: 1 [13/96 (14%)]	Loss: 44.015282
Train Epoche: 1 [14/96 (15%)]	Loss: 465.076630
Train Epoche: 1 [15/96 (16%)]	Loss: 269.214050
Train Epoche: 1 [16/96 (17%)]	Loss: 376.310120
Train Epoche: 1 [17/96 (18%)]	Loss: 152.978638
Train Epoche: 1 [18/96 (19%)]	Loss: 199.133240
Train Epoche: 1 [19/96 (20%)]	Loss: 299.566803
Train Epoche: 1 [20/96 (21%)]	Loss: 317.942230
Train Epoche: 1 [21/96 (22%)]	Loss: 0.140721
T

Train Epoche: 1 [177/96 (184%)]	Loss: 2.665544
Train Epoche: 1 [178/96 (185%)]	Loss: 243.600906
Train Epoche: 1 [179/96 (186%)]	Loss: 39.472134
Train Epoche: 1 [180/96 (188%)]	Loss: 66.639198
Train Epoche: 1 [181/96 (189%)]	Loss: 25.533787
Train Epoche: 1 [182/96 (190%)]	Loss: 17.761419
Train Epoche: 1 [183/96 (191%)]	Loss: 26.186115
Train Epoche: 1 [184/96 (192%)]	Loss: 6.071060
Train Epoche: 1 [185/96 (193%)]	Loss: 0.806710
Train Epoche: 1 [186/96 (194%)]	Loss: 48.251728
Train Epoche: 1 [187/96 (195%)]	Loss: 23.962925
Train Epoche: 1 [188/96 (196%)]	Loss: 230.546066
Train Epoche: 1 [189/96 (197%)]	Loss: 49.862743
Train Epoche: 1 [190/96 (198%)]	Loss: 169.132980
Train Epoche: 1 [191/96 (199%)]	Loss: 0.005661
Train Epoche: 1 [192/96 (200%)]	Loss: 2.945440
Train Epoche: 1 [193/96 (201%)]	Loss: 14.407281
Train Epoche: 1 [194/96 (202%)]	Loss: 50.130733
Train Epoche: 1 [195/96 (203%)]	Loss: 69.813828
Train Epoche: 1 [196/96 (204%)]	Loss: 302.109863
Train Epoche: 1 [197/96 (205%)]	Loss: 22.

Train Epoche: 1 [353/96 (368%)]	Loss: 12.850769
Train Epoche: 1 [354/96 (369%)]	Loss: 21.378311
Train Epoche: 1 [355/96 (370%)]	Loss: 112.362152
Train Epoche: 1 [356/96 (371%)]	Loss: 197.513168
Train Epoche: 1 [357/96 (372%)]	Loss: 114.378860
Train Epoche: 1 [358/96 (373%)]	Loss: 22.365273
Train Epoche: 1 [359/96 (374%)]	Loss: 97.926552
Train Epoche: 1 [360/96 (375%)]	Loss: 6.252251
Train Epoche: 1 [361/96 (376%)]	Loss: 60.935226
Train Epoche: 1 [362/96 (377%)]	Loss: 74.922905
Train Epoche: 1 [363/96 (378%)]	Loss: 16.448647
Train Epoche: 1 [364/96 (379%)]	Loss: 13.101506
Train Epoche: 1 [365/96 (380%)]	Loss: 152.837097
Train Epoche: 1 [366/96 (381%)]	Loss: 10.044191
Train Epoche: 1 [367/96 (382%)]	Loss: 1.890057
Train Epoche: 1 [368/96 (383%)]	Loss: 24.319288
Train Epoche: 1 [369/96 (384%)]	Loss: 20.625067
Train Epoche: 1 [370/96 (385%)]	Loss: 225.093567
Train Epoche: 1 [371/96 (386%)]	Loss: 69.194374
Train Epoche: 1 [372/96 (388%)]	Loss: 6.365253
Train Epoche: 1 [373/96 (389%)]	Loss: 

Train Epoche: 1 [548/96 (571%)]	Loss: 56.474571
Train Epoche: 1 [549/96 (572%)]	Loss: 4.940430
Train Epoche: 1 [550/96 (573%)]	Loss: 14.735056
Train Epoche: 1 [551/96 (574%)]	Loss: 13.385953
Train Epoche: 1 [552/96 (575%)]	Loss: 10.762835
Train Epoche: 1 [553/96 (576%)]	Loss: 0.106908
Train Epoche: 1 [554/96 (577%)]	Loss: 12.545610
Train Epoche: 1 [555/96 (578%)]	Loss: 8.251269
Train Epoche: 1 [556/96 (579%)]	Loss: 10.156180
Train Epoche: 1 [557/96 (580%)]	Loss: 16.668320
Train Epoche: 1 [558/96 (581%)]	Loss: 4.012468
Train Epoche: 1 [559/96 (582%)]	Loss: 77.385681
Train Epoche: 1 [560/96 (583%)]	Loss: 5.191860
Train Epoche: 1 [561/96 (584%)]	Loss: 1.039693
Train Epoche: 1 [562/96 (585%)]	Loss: 0.828935
Train Epoche: 1 [563/96 (586%)]	Loss: 0.338112
Train Epoche: 1 [564/96 (588%)]	Loss: 9.225192
Train Epoche: 1 [565/96 (589%)]	Loss: 0.018839
Train Epoche: 1 [566/96 (590%)]	Loss: 12.952704
Train Epoche: 1 [567/96 (591%)]	Loss: 30.392670
Train Epoche: 1 [568/96 (592%)]	Loss: 49.315334
Tr

...

Train Epoche: 3 [1277/96 (1330%)]	Loss: 5.984699
Train Epoche: 3 [1278/96 (1331%)]	Loss: 0.190964
Train Epoche: 3 [1279/96 (1332%)]	Loss: 0.339852
Train Epoche: 3 [1280/96 (1333%)]	Loss: 31.815592
Train Epoche: 3 [1281/96 (1334%)]	Loss: 0.685148
Train Epoche: 3 [1282/96 (1335%)]	Loss: 0.567920
Train Epoche: 3 [1283/96 (1336%)]	Loss: 12.627066
Train Epoche: 3 [1284/96 (1338%)]	Loss: 0.432638
Train Epoche: 3 [1285/96 (1339%)]	Loss: 2.727305
Train Epoche: 3 [1286/96 (1340%)]	Loss: 0.111325
Train Epoche: 3 [1287/96 (1341%)]	Loss: 5.003300
Train Epoche: 3 [1288/96 (1342%)]	Loss: 0.335340
Train Epoche: 3 [1289/96 (1343%)]	Loss: 2.499052
Train Epoche: 3 [1290/96 (1344%)]	Loss: 7.633105
Train Epoche: 3 [1291/96 (1345%)]	Loss: 4.482079
Train Epoche: 3 [1292/96 (1346%)]	Loss: 3.912131
Train Epoche: 3 [1293/96 (1347%)]	Loss: 26.495970
Train Epoche: 3 [1294/96 (1348%)]	Loss: 2.070867
Train Epoche: 3 [1295/96 (1349%)]	Loss: 7.531113
Train Epoche: 3 [1296/96 (1350%)]	Loss: 0.443556
Train Epoche: 3 [

Train Epoche: 3 [1443/96 (1503%)]	Loss: 0.040799
Train Epoche: 3 [1444/96 (1504%)]	Loss: 1.624463
Train Epoche: 3 [1445/96 (1505%)]	Loss: 6.399389
Train Epoche: 3 [1446/96 (1506%)]	Loss: 1.553715
Train Epoche: 3 [1447/96 (1507%)]	Loss: 3.566165
Train Epoche: 3 [1448/96 (1508%)]	Loss: 231.892334
Train Epoche: 3 [1449/96 (1509%)]	Loss: 3.356291
Train Epoche: 3 [1450/96 (1510%)]	Loss: 3.538827
Train Epoche: 3 [1451/96 (1511%)]	Loss: 5.192248
Train Epoche: 3 [1452/96 (1512%)]	Loss: 0.024759
Train Epoche: 3 [1453/96 (1514%)]	Loss: 91.804169
Train Epoche: 3 [1454/96 (1515%)]	Loss: 0.448534
Train Epoche: 3 [1455/96 (1516%)]	Loss: 0.244318
Train Epoche: 3 [1456/96 (1517%)]	Loss: 2.097049
Train Epoche: 3 [1457/96 (1518%)]	Loss: 2.098794
Train Epoche: 3 [1458/96 (1519%)]	Loss: 5.485961
Train Epoche: 3 [1459/96 (1520%)]	Loss: 1.040104
Train Epoche: 3 [1460/96 (1521%)]	Loss: 5.220003
Train Epoche: 3 [1461/96 (1522%)]	Loss: 15.666691
Train Epoche: 3 [1462/96 (1523%)]	Loss: 1.694251
Train Epoche: 3 

Train Epoche: 3 [1613/96 (1680%)]	Loss: 1.049830
Train Epoche: 3 [1614/96 (1681%)]	Loss: 8.259188
Train Epoche: 3 [1615/96 (1682%)]	Loss: 0.110289
Train Epoche: 3 [1616/96 (1683%)]	Loss: 15.771244
Train Epoche: 3 [1617/96 (1684%)]	Loss: 16.418221
Train Epoche: 3 [1618/96 (1685%)]	Loss: 0.353472
Train Epoche: 3 [1619/96 (1686%)]	Loss: 19.073025
Train Epoche: 3 [1620/96 (1688%)]	Loss: 23.805580
Train Epoche: 3 [1621/96 (1689%)]	Loss: 0.210161
Train Epoche: 3 [1622/96 (1690%)]	Loss: 3.818677
Train Epoche: 3 [1623/96 (1691%)]	Loss: 0.364039
Train Epoche: 3 [1624/96 (1692%)]	Loss: 6.582472
Train Epoche: 3 [1625/96 (1693%)]	Loss: 10.769957
Train Epoche: 3 [1626/96 (1694%)]	Loss: 4.369391
Train Epoche: 3 [1627/96 (1695%)]	Loss: 63.166241
Train Epoche: 3 [1628/96 (1696%)]	Loss: 12.923668
Train Epoche: 3 [1629/96 (1697%)]	Loss: 104.485443
Train Epoche: 3 [1630/96 (1698%)]	Loss: 0.053820
Train Epoche: 3 [1631/96 (1699%)]	Loss: 56.067875
Train Epoche: 3 [1632/96 (1700%)]	Loss: 3.792563
Train Epoc

Train Epoche: 3 [1783/96 (1857%)]	Loss: 1.634150
Train Epoche: 3 [1784/96 (1858%)]	Loss: 6.329586
Train Epoche: 3 [1785/96 (1859%)]	Loss: 2.858719
Train Epoche: 3 [1786/96 (1860%)]	Loss: 2.980958
Train Epoche: 3 [1787/96 (1861%)]	Loss: 0.083853
Train Epoche: 3 [1788/96 (1862%)]	Loss: 1.650233
Train Epoche: 3 [1789/96 (1864%)]	Loss: 0.003599
Train Epoche: 3 [1790/96 (1865%)]	Loss: 45.892670
Train Epoche: 3 [1791/96 (1866%)]	Loss: 2.686808
Train Epoche: 3 [1792/96 (1867%)]	Loss: 8.590115
Train Epoche: 3 [1793/96 (1868%)]	Loss: 0.065229
Train Epoche: 3 [1794/96 (1869%)]	Loss: 3.783356
Train Epoche: 3 [1795/96 (1870%)]	Loss: 3.274040
Train Epoche: 3 [1796/96 (1871%)]	Loss: 1.954435
Train Epoche: 3 [1797/96 (1872%)]	Loss: 2.426826
Train Epoche: 3 [1798/96 (1873%)]	Loss: 0.025016
Train Epoche: 3 [1799/96 (1874%)]	Loss: 3.169960
Train Epoche: 3 [1800/96 (1875%)]	Loss: 1.292671
Train Epoche: 3 [1801/96 (1876%)]	Loss: 3.360541
Train Epoche: 3 [1802/96 (1877%)]	Loss: 20.159576
Train Epoche: 3 [1

Train Epoche: 3 [1950/96 (2031%)]	Loss: 0.368623
Train Epoche: 3 [1951/96 (2032%)]	Loss: 0.589324
Train Epoche: 3 [1952/96 (2033%)]	Loss: 4.123941
Train Epoche: 3 [1953/96 (2034%)]	Loss: 47.057053
Train Epoche: 3 [1954/96 (2035%)]	Loss: 1.358780
Train Epoche: 3 [1955/96 (2036%)]	Loss: 14.653416
Train Epoche: 3 [1956/96 (2038%)]	Loss: 3.872564
Train Epoche: 3 [1957/96 (2039%)]	Loss: 3.552732
Train Epoche: 3 [1958/96 (2040%)]	Loss: 3.562769
Train Epoche: 3 [1959/96 (2041%)]	Loss: 3.344995
Train Epoche: 3 [1960/96 (2042%)]	Loss: 1.794782
Train Epoche: 3 [1961/96 (2043%)]	Loss: 17.386757
Train Epoche: 3 [1962/96 (2044%)]	Loss: 2.440611
Train Epoche: 3 [1963/96 (2045%)]	Loss: 0.023076
Train Epoche: 3 [1964/96 (2046%)]	Loss: 4.464834
Train Epoche: 3 [1965/96 (2047%)]	Loss: 5.564886
Train Epoche: 3 [1966/96 (2048%)]	Loss: 9.238482
Train Epoche: 3 [1967/96 (2049%)]	Loss: 0.258056
Train Epoche: 3 [1968/96 (2050%)]	Loss: 3.918486
Train Epoche: 3 [1969/96 (2051%)]	Loss: 3.607605
Train Epoche: 3 [

In [10]:
result_dfs = {}
for raceId, dict in total_results.items():    
    #Auswerten der Vorhersagen aus den Outputdaten des Modells
    A = [x[0][0] for x in list((dict.values()))]
    A_dyn = [x[0][0] for x in list((total_results_dynamic[raceId].values()))]
    y = list(dict.keys())
    y_dyn = list(total_results_dynamic[raceId].keys())
    t = pd.DataFrame(columns = ['target', 'prediction'])
    t_dyn = pd.DataFrame(columns = ['target', 'prediction'])
    t['target'] = y
    t['prediction'] = A
    t_dyn['target'] = y_dyn
    t_dyn['prediction'] = A_dyn
    #print("raceId", raceId)
    temp = sliced_races[raceId]
    #name = split_by_race[raceId].where()
    #sortieren des DataFrames nach den vorhergesagten Positionen, aufsteigend
    end = sqldf.sqldf('''select * from t order by prediction ASC''')
    end.reset_index(inplace = True)
    end_dyn = sqldf.sqldf('''select * from t_dyn order by prediction ASC''')
    end_dyn.reset_index(inplace = True)
    #Da DF nun nach prediction aufsteigend sortiert ist, kann veränderter Index als predictete Position gesetzt werden
    end.rename(columns = {'index': 'predicted_position'},inplace = True)
    end_dyn.rename(columns = {'index': 'predicted_position'},inplace = True)
    #zur Übersichtlichkeit wird nun der endgültige DF nach den richtigen Positionen (target) sortiert (aufsteigend)
    end = sqldf.sqldf('''select * from end order by target ASC''')
    end_dyn = sqldf.sqldf('''select * from end_dyn order by target ASC''')
    end['predicted_position'] = end['predicted_position']+1
    end_dyn['predicted_position'] = end_dyn['predicted_position']+1
    #pred_name = sqldf.sqldf("""select 
    #            distinct
    #            d.driver_fullname as name,
    #            t.predicted_position as pred,
    #            d.podium_position as target_position
    #            from 
    #            end t inner join temp d
    #            on t.predicted_position = d.podium_position""")
    end["driver_pred"] = 0
    end["driver_target"] = 0
    end_dyn["driver_pred"] = 0
    end_dyn["driver_target"] = 0
    for pos in temp.podium_position.unique():
        
        name = temp.where(temp.podium_position == pos).dropna(how = "all")
        #print(name)
        name_ = name["driver_fullname"][list(name.index)[0]]
        idx_target = end.where(end.target == pos).dropna(how = "all").index
        idx_pred = end.where(end.predicted_position == pos).dropna(how = "all").index
        end.loc[idx_target,"driver_target"] = name_
        end.loc[idx_pred,"driver_pred"] = name_
        
        idx_target = end_dyn.where(end_dyn.target == pos).dropna(how = "all").index
        idx_pred = end_dyn.where(end_dyn.predicted_position == pos).dropna(how = "all").index
        end_dyn.loc[idx_target,"driver_target"] = name_
        end_dyn.loc[idx_pred,"driver_pred"] = name_
        
    #umstellen der Spaltenreihenfolge
    '''end = end[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction',"driver_target", "driver_pred"]]'''
    end = end[['target', 'predicted_position', 'prediction']]
    end_dyn = end_dyn[['target', 'predicted_position', 'prediction']]
    result_dfs[raceId] = end
    r = str(raceId)+'_dynamic'
    result_dfs[r] = end_dyn
    
for key, value in result_dfs.items():
    print('RaceId:',key)
    print(value.head(10),'\n')

RaceId: 917
   target  predicted_position  prediction
0     1.0                   1    1.873325
1     2.0                   2    1.968295
2     3.0                   3    3.868555
3     4.0                   6    5.955686
4     5.0                   7    7.043250
5     6.0                   8    8.678005
6     7.0                   4    5.437355
7     8.0                   5    5.471093
8    10.0                  10   10.900343
9    11.0                   9    9.947760 

RaceId: 917_dynamic
   target  predicted_position  prediction
0     1.0                   1    1.840542
1     2.0                   2    1.960681
2     3.0                   3    3.457597
3     4.0                   6    5.455589
4     5.0                   7    6.487725
5     6.0                   8    7.748151
6     7.0                   4    4.843204
7     8.0                   5    4.845038
8    10.0                  10    9.822279
9    11.0                   9    8.584066 

RaceId: 977
   target  predicted_positio